In [74]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

In [75]:
shot_metrics_df = pd.read_csv("shot_metric_df.csv")
shot_metrics_df.head()

,Unnamed: 0,event_id,event_index,period,timestamp,minute,second,possession,duration,obv_for_after,obv_for_before,obv_for_net,obv_against_after,obv_against_before,obv_against_net,obv_total_net,under_pressure,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,location_x,location_y,location_z,related_events,player_id,player_name,position_id,position_name,statsbomb_xg,goal,penalty,body_part,opposition_in_triangle,gameId,match_date,Count HI_now,Count HI_half,Count HI_game,Count HSR_now,Count HSR_half,Count HSR_game,Count High Acceleration_now,Count High Acceleration_half,Count High Acceleration_game,Count High Deceleration_now,Count High Deceleration_half,Count High Deceleration_game,Count Medium Acceleration_now,Count Medium Acceleration_half,Count Medium Acceleration_game,Count Medium Deceleration_now,Count Medium Deceleration_half,Count Medium Deceleration_game,Count Sprint_now,Count Sprint_half,Count Sprint_game,High Intensity (HI) Distance_now,High Intensity (HI) Distance_half,High Intensity (HI) Distance_game,High Speed Running (HSR) Distance_now,High Speed Running (HSR) Distance_half,High Speed Running (HSR) Distance_game,M/min_now,M/min_half,M/min_game,Max Speed_now,Max Speed_half,Max Speed_game,Running Distance_now,Running Distance_half,Running Distance_game,Sprinting Distance_now,Sprinting Distance_half,Sprinting Distance_game,Total Distance_now,Total Distance_half,Total Distance_game,Count HI_1_week,Count HSR_1_week,Count High Acceleration_1_week,Count High Deceleration_1_week,Count Medium Acceleration_1_week,Count Medium Deceleration_1_week,Count Sprint_1_week,High Intensity (HI) Distance_1_week,High Speed Running (HSR) Distance_1_week,M/min_1_week,Max Speed_1_week,Running Distance_1_week,Sprinting Distance_1_week,Total Distance_1_week,Count HI_2_week,Count HSR_2_week,Count High Acceleration_2_week,Count High Deceleration_2_week,Count Medium Acceleration_2_week,Count Medium Deceleration_2_week,Count Sprint_2_week,High Intensity (HI) Distance_2_week,High Speed Running (HSR) Distance_2_week,M/min_2_week,Max Speed_2_week,Running Distance_2_week,Sprinting Distance_2_week,Total Distance_2_week,Count HI_1_month,Count HSR_1_month,Count High Acceleration_1_month,Count High Deceleration_1_month,Count Medium Acceleration_1_month,Count Medium Deceleration_1_month,Count Sprint_1_month,High Intensity (HI) Distance_1_month,High Speed Running (HSR) Distance_1_month,M/min_1_month,Max Speed_1_month,Running Distance_1_month,Sprinting Distance_1_month,Total Distance_1_month
0,0,24392d1d-314a-4063-a729-7775bc0a2c45,130.0,1.0,00:03:04.524,3.0,4.0,9.0,0.471596,0.306241,0.278243,0.027997,0.007197,0.007197,0.0,0.027997,True,16.0,Shot,36.0,Manchester City,2.0,From Corner,36.0,Manchester City,112.7,39.9,0.22,"67dc6041-d315-4ebb-ac8e-18f3fcd24abc, 7d4f2f91...",23616.0,Erling Håland,23.0,Center Forward,0.166126,1.0,0.0,Left Foot,1.0,3892755,2023-08-11,1.200000,0.0,0.0,1.000000,0.0,0.0,0.200000,0.0,0.0,0.400000,0.0,0.0,6.800000,0.0,0.0,4.400000,0.0,0.0,0.200000,0.0,0.0,17.017352,0.000000,0.000000,15.196193,0.000000,0.000000,20.433466,0.000000,0.000000,5.187833,0.000000,0.000000,37.352316,0.000000,0.000000,1.821159,0.000000,0.000000,306.501986,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3c527d7b-6932-4190-8424-d1d3dcdd6a4d,191.0,1.0,00:05:11.529,5.0,11.0,11.0,1.023995,0.000000,0.034124,-0.034124,0.009421,0.009421,0.0,-0.034124,NaN,16.0,Shot,36.0,Manchester City,7.0,From Goal Kick,36.0,Manchester City,88.4,46.0,0.00,207b5c5a-ff48-4534-a5e2-0f83b20df80a,3089.0,Kevin De Bruyne,13.0,Right Center Midfield,0.023693,0.0,0.0,Right Foot,2.0,3892755,2023-08-11,4.000000,0.0,0.0,3.666667,0.0,0.0,0.666667,0.0,0.0,1.000000,0.0,0.0,14.000000,0.0,0.0,11.000000,0.0,0.0,0.333333,0.0,0.0,37.031555,0.000000,0.000000,31.198649,0.000000,0.000000,40.155532,0.000000,0.00000

In [76]:
#clean df for ML use
shot_metrics_df =shot_metrics_df[shot_metrics_df.penalty==0]
shot_metrics_df = shot_metrics_df.fillna(0)
shot_metrics_df.under_pressure = np.where(shot_metrics_df.under_pressure==True,1,0)
shot_metrics_df.under_pressure

0        1
1        0
2        0
3        0
4        0
        ..
61649    1
61650    1
61651    0
61652    1
61653    1
Name: under_pressure, Length: 60896, dtype: int64

In [77]:
#leave spanish clubs out
#clubs to leave out of training

test_clubs = [
    "Athletic Club",
    "Getafe",
    "Real Betis",
    "Girona",
  "Las Palmas",
    "Sevilla",
    "Celta Vigo",
   "Deportivo Alavés",
     "Osasuna",
     "Barcelona",
    "Valencia",
     "Rayo Vallecano",
    "Real Sociedad",
     "Real Madrid",
     "Mallorca",
    "Villarreal",
    "Atlético Madrid"
    ]

In [78]:
shot_metrics_df.head(1)

,Unnamed: 0,event_id,event_index,period,timestamp,minute,second,possession,duration,obv_for_after,obv_for_before,obv_for_net,obv_against_after,obv_against_before,obv_against_net,obv_total_net,under_pressure,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,location_x,location_y,location_z,related_events,player_id,player_name,position_id,position_name,statsbomb_xg,goal,penalty,body_part,opposition_in_triangle,gameId,match_date,Count HI_now,Count HI_half,Count HI_game,Count HSR_now,Count HSR_half,Count HSR_game,Count High Acceleration_now,Count High Acceleration_half,Count High Acceleration_game,Count High Deceleration_now,Count High Deceleration_half,Count High Deceleration_game,Count Medium Acceleration_now,Count Medium Acceleration_half,Count Medium Acceleration_game,Count Medium Deceleration_now,Count Medium Deceleration_half,Count Medium Deceleration_game,Count Sprint_now,Count Sprint_half,Count Sprint_game,High Intensity (HI) Distance_now,High Intensity (HI) Distance_half,High Intensity (HI) Distance_game,High Speed Running (HSR) Distance_now,High Speed Running (HSR) Distance_half,High Speed Running (HSR) Distance_game,M/min_now,M/min_half,M/min_game,Max Speed_now,Max Speed_half,Max Speed_game,Running Distance_now,Running Distance_half,Running Distance_game,Sprinting Distance_now,Sprinting Distance_half,Sprinting Distance_game,Total Distance_now,Total Distance_half,Total Distance_game,Count HI_1_week,Count HSR_1_week,Count High Acceleration_1_week,Count High Deceleration_1_week,Count Medium Acceleration_1_week,Count Medium Deceleration_1_week,Count Sprint_1_week,High Intensity (HI) Distance_1_week,High Speed Running (HSR) Distance_1_week,M/min_1_week,Max Speed_1_week,Running Distance_1_week,Sprinting Distance_1_week,Total Distance_1_week,Count HI_2_week,Count HSR_2_week,Count High Acceleration_2_week,Count High Deceleration_2_week,Count Medium Acceleration_2_week,Count Medium Deceleration_2_week,Count Sprint_2_week,High Intensity (HI) Distance_2_week,High Speed Running (HSR) Distance_2_week,M/min_2_week,Max Speed_2_week,Running Distance_2_week,Sprinting Distance_2_week,Total Distance_2_week,Count HI_1_month,Count HSR_1_month,Count High Acceleration_1_month,Count High Deceleration_1_month,Count Medium Acceleration_1_month,Count Medium Deceleration_1_month,Count Sprint_1_month,High Intensity (HI) Distance_1_month,High Speed Running (HSR) Distance_1_month,M/min_1_month,Max Speed_1_month,Running Distance_1_month,Sprinting Distance_1_month,Total Distance_1_month
0,0,24392d1d-314a-4063-a729-7775bc0a2c45,130.0,1.0,00:03:04.524,3.0,4.0,9.0,0.471596,0.306241,0.278243,0.027997,0.007197,0.007197,0.0,0.027997,1,16.0,Shot,36.0,Manchester City,2.0,From Corner,36.0,Manchester City,112.7,39.9,0.22,"67dc6041-d315-4ebb-ac8e-18f3fcd24abc, 7d4f2f91...",23616.0,Erling Håland,23.0,Center Forward,0.166126,1.0,0.0,Left Foot,1.0,3892755,2023-08-11,1.2,0.0,0.0,1.0,0.0,0.0,0.2,0.0,0.0,0.4,0.0,0.0,6.8,0.0,0.0,4.4,0.0,0.0,0.2,0.0,0.0,17.017352,0.0,0.0,15.196193,0.0,0.0,20.433466,0.0,0.0,5.187833,0.0,0.0,37.352316,0.0,0.0,1.821159,0.0,0.0,306.501986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
ML_data= shot_metrics_df[["team_name","goal", "under_pressure","play_pattern_name","body_part" ,"location_x","location_y","location_z","opposition_in_triangle"]]
play_pattern_onehot = pd.get_dummies(ML_data.play_pattern_name)#, dtype=float)
body_part_onehot = pd.get_dummies(ML_data.body_part)#, dtype=float)
ML_data = pd.concat([ML_data,play_pattern_onehot,body_part_onehot], axis=1).drop(["play_pattern_name","body_part"],axis=1)
training_data =  ML_data[~ML_data.team_name.isin(test_clubs)].drop("team_name",axis=1)
test_data =  ML_data[ML_data.team_name.isin(test_clubs)].drop("team_name",axis=1)

In [101]:
X_train_raw = training_data.drop("goal", axis=1).astype(float)
y_train_raw = training_data["goal"].astype(int)

X_test = np.array(test_data.drop("goal", axis=1).astype(float))
y_test = np.array(test_data["goal"].astype(int))

#scaler = StandardScaler()
#model = scaler.fit(X_train_raw)
#X_scaled = model.transform(X_train_raw)
#X_test = model.transform(X_test_raw)
X_train, X_val, y_train, y_val = train_test_split(
    np.array(X_train_raw), np.array(y_train_raw), test_size=0.25, random_state=123
)

In [110]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print(X_train_res.shape, y_train_res.shape)  # balanced now

(69330, 18) (69330,)


In [114]:
np.bincount(y_train_res)

array([34665, 34665])

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score


# Define features and target
neg, pos = np.bincount(y_train)
scale = neg / pos
print(f"scale_pos_weight = {scale:.2f}")


# Initialize XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=1000,       # number of trees
    max_depth=5,           # depth of each tree
    learning_rate=0.05,     # step size shrinkage
    subsample=0.8,        # row sampling
    colsample_bytree=0.8,  # feature sampling
    random_state=42,
    use_label_encoder=False,
    #objective="binary:logitraw",
    eval_metric="logloss",
    #scale_pos_weight=4  # avoids warning about label encoder
)

# Train model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]

# Evaluation
acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC: {roc:.3f}")

scale_pos_weight = 8.43


c:\Users\conny\OneDrive\Documents\Projects\Hudl_Performance\Hudl_Performance_Insights_2025\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:09:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.904
ROC AUC: 0.757


In [170]:
statsbomb_test = np.where(shot_metrics_df[shot_metrics_df.team_name.isin(test_clubs)].statsbomb_xg>0.5,1,0)
statsbomb_actual = np.array(shot_metrics_df[shot_metrics_df.team_name.isin(test_clubs)].goal)
statsbomb_proba = np.array(shot_metrics_df[shot_metrics_df.team_name.isin(test_clubs)].statsbomb_xg)

In [171]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[8178,   83],
       [ 799,  131]])

In [172]:
confusion_matrix(statsbomb_actual,statsbomb_test )

array([[8206,   55],
       [ 794,  136]])

In [173]:
acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC: {roc:.3f}")

Accuracy: 0.904
ROC AUC: 0.757


In [174]:
acc = accuracy_score(statsbomb_actual,statsbomb_test )
roc = roc_auc_score(statsbomb_actual, statsbomb_proba)

print(f"Accuracy: {acc:.3f}")
print(f"ROC AUC: {roc:.3f}")

Accuracy: 0.908
ROC AUC: 0.793


In [ ]:
pip install autogluon

In [ ]:
from autogluon.tabular import TabularPredictor
import numpy as np

# Compute class weights
neg, pos = np.bincount(y_train)
scale = neg / pos

# Assign weights: higher weight for goals (minority class)
weights = np.where(y_train == 1, scale, 1.0)

# Add weights column to training data
train_with_weights = training_data.copy()
train_with_weights["weight"] = weights

# Fit AutoGluon with weights
predictor = TabularPredictor(
    label="goal",
    eval_metric="f1",       # better than accuracy for imbalanced classes
    problem_type="binary"
).fit(
    train_with_weights,
    sample_weight="weight",
    presets="best_quality"
)